In [1]:
import numpy as np
import cv2
import os

# Path to the folder containing your JPEG images
folder_path = 'TrainingDataset\\'
# CHANGE THE PATH ABOVE


# Loop over the JPEG images in the folder and read them into a NumPy array
dataset = []
clearName = []
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):
        clearName.append(filename)
        image = cv2.imread(os.path.join(folder_path, filename))
        dataset.append(image)

# Convert the list of images to a NumPy array
dataset = np.array(dataset)


C:\Users\facep\AppData\Local\Temp\ipykernel_5128\1159008003.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset = np.array(dataset)


In [3]:
len(dataset)

2000

In [4]:
def add_speckle_noise(image):
    # Add speckle noise to each color channel
    row, col, ch = image.shape
    gauss = np.random.randn(row, col, ch)
    noisy = image + image * gauss
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

In [5]:
def add_gaussian_noise(image, var):
    # Add Gaussian noise
    row, col, ch = image.shape
    mean = 0
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy = image + gauss
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

In [6]:
def add_salt_pepper_noise(image, amount):
    # Add salt and pepper noise
    s_vs_p = 0.5
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
            for i in image.shape]
    out[coords] = 1
    # Pepper mode
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
            for i in image.shape]
    out[coords] = 0
    return out

In [7]:
def add_poisson_noise(image, scale):
    out = np.random.poisson(image * scale) / float(scale)
    out = np.clip(out, 0, 255)
    return out

In [8]:
def segment(img):

    # Reshape image into a 2D array of pixels
    pixels = img.reshape((-1, 3))

    # Convert data type to float32 for k-means algorithm
    pixels = np.float32(pixels)

    # Define k-means parameters
    k = 3  # Number of clusters
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

    # Run k-means algorithm
    ret, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Convert centers from float32 to uint8 for displaying image
    centers = np.uint8(centers)

    # Assign each pixel to its corresponding cluster center
    segmented_img = centers[labels.flatten()]

    # Reshape segmented image back to original dimensions
    segmented_img = segmented_img.reshape(img.shape)
    
    return segmented_img

In [9]:
# Define the rotation parameters
min_angle = -30
max_angle = 30

# Define the color shift parameters
min_intensity = -100
max_intensity = 100

In [10]:
# Create empty directories to store the noisy images
folders_for_new_pics = ["poissonNoise", "spNoise", "speckleNoise", "hist", 
                        "gaussianNoise", "flipped", "rotate", "colorShifting", "gaussianBlur",
                        "brightness", "contrast", "saturation", "segmented"]

for folder in folders_for_new_pics:
    os.makedirs(f"{os.getcwd()}\\enhancedDataset\\{folder}", exist_ok=True)

os.chdir("enhancedDataset\\")

In [15]:
# Loop over the dataset and add different types of noise to each image
for i, image in enumerate(dataset):
    
        
    # SPECKLE
    noisy_image = add_speckle_noise(image)
    cv2.imwrite(f'speckleNoise/{clearName[i].split(".")[0]}_speckle.jpg', noisy_image)
    
    
    # GAUSSIAN
    noisy_image = add_gaussian_noise(image, var=5000)
    cv2.imwrite(f'gaussianNoise/{clearName[i].split(".")[0]}_gaussian5.jpg', noisy_image)  
    
    noisy_image = add_gaussian_noise(image, var=10000)
    cv2.imwrite(f'gaussianNoise/{clearName[i].split(".")[0]}_gaussian10.jpg', noisy_image)  
    
    noisy_image = add_gaussian_noise(image, var=15000)
    cv2.imwrite(f'gaussianNoise/{clearName[i].split(".")[0]}_gaussian15.jpg', noisy_image)
    
    
    # SALT PEPPER
    noisy_image = add_salt_pepper_noise(image, amount=0.00001)
    cv2.imwrite(f'spNoise/{clearName[i].split(".")[0]}_sp00001.jpg', noisy_image)
    
    noisy_image = add_salt_pepper_noise(image, amount=0.000001)
    cv2.imwrite(f'spNoise/{clearName[i].split(".")[0]}_sp000001.jpg', noisy_image)
    
    noisy_image = add_salt_pepper_noise(image, amount=0.000005)
    cv2.imwrite(f'spNoise/{clearName[i].split(".")[0]}_sp000005.jpg', noisy_image)
         
    # POISSON
    noisy_image = add_poisson_noise(image, scale=0.05)
    cv2.imwrite(f'poissonNoise/{clearName[i].split(".")[0]}_poisson05.jpg', noisy_image)

    noisy_image = add_poisson_noise(image, scale=0.001)
    cv2.imwrite(f'poissonNoise/{clearName[i].split(".")[0]}_poisson001.jpg', noisy_image)
    
    noisy_image = add_poisson_noise(image, scale=0.005)
    cv2.imwrite(f'poissonNoise/{clearName[i].split(".")[0]}_poisson005.jpg', noisy_image)
    
    
    # FLIP
    cv2.imwrite(f'flipped/{clearName[i].split(".")[0]}_flip.jpg', cv2.flip(image, 1))


    # ROTATION
    # Generate random rotation angle
    angle = np.random.uniform(min_angle, max_angle)
    
    # Rotate the image
    height, width, _ = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

    # Save the rotated image
    cv2.imwrite(f'rotate/{clearName[i].split(".")[0]}_rotate.jpg', rotated_image)


    # COLOR SHIFTING
    # Generate random color shift values
    r_shift = np.random.randint(min_intensity, max_intensity + 1)
    g_shift = np.random.randint(min_intensity, max_intensity + 1)
    b_shift = np.random.randint(min_intensity, max_intensity + 1)

    # Shift the color channels
    shifted_image = np.zeros_like(image)
    shifted_image[:, :, 0] = np.clip(image[:, :, 0] + b_shift, 0, 255)
    shifted_image[:, :, 1] = np.clip(image[:, :, 1] + g_shift, 0, 255)
    shifted_image[:, :, 2] = np.clip(image[:, :, 2] + r_shift, 0, 255)

    # Save the color-shifted image
    cv2.imwrite(f'colorShifting/{clearName[i].split(".")[0]}_shiftedColor.jpg', shifted_image)
    
    
    # HISTOGRAM EQUALIZATION
    # Apply histogram equalization to the image
    equalized_image = cv2.equalizeHist(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    equalized_image = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2BGR)

    # Save the equalized image
    cv2.imwrite(f'hist/{clearName[i].split(".")[0]}_histEq.jpg', equalized_image)
    
    
    # GAUSSIAN BLUR
    # Apply Gaussian blur with kernel size 25x25 and sigma value of 100
    blur = cv2.GaussianBlur(image, (25, 25), 100)
    
    # Save the blurred image
    cv2.imwrite(f'gaussianBlur/{clearName[i].split(".")[0]}_gaussianBlur.jpg', blur)


    # BRIGHTNESS
    # Increase brightness by 100
    bright = cv2.add(image, 100)

    # Decrease brightness by 100
    dark = cv2.subtract(image, 100)
    
    # Save the bright and dark images 
    cv2.imwrite(f'brightness/{clearName[i].split(".")[0]}_bright100.jpg', bright)
    cv2.imwrite(f'brightness/{clearName[i].split(".")[0]}_dark100.jpg', dark)


    # CONTRAST
    # Increase contrast by 1.5
    high_contrast = cv2.convertScaleAbs(image, alpha=1.5, beta=0)

    # Decrease contrast by 0.5
    low_contrast = cv2.convertScaleAbs(image, alpha=0.5, beta=0)
    
    # Save the high and low contrast images 
    cv2.imwrite(f'contrast/{clearName[i].split(".")[0]}_highContrast.jpg', high_contrast)
    cv2.imwrite(f'contrast/{clearName[i].split(".")[0]}_lowContrast.jpg', low_contrast)


    # COLOR SATURATION
    # Convert image from BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Increase saturation by 50%
    hsv[:,:,1] = np.clip(hsv[:,:,1]*1.9, 0, 255).astype(np.uint8)
    
    # Convert image back to BGR
    sat = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    # Save the saturated image 
    cv2.imwrite(f'saturation/{clearName[i].split(".")[0]}_saturation.jpg', sat)
    
    
    # SEGMENTATION
    segmented_image = segment(image)
    cv2.imwrite(f'segmented/{clearName[i].split(".")[0]}_segmented.jpg', segmented_image)

C:\Users\facep\AppData\Local\Temp\ipykernel_5128\1180357195.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[coords] = 1
C:\Users\facep\AppData\Local\Temp\ipykernel_5128\1180357195.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[coords] = 0
